In [82]:
import os
import pandas as pd
import fnmatch
import re

In [83]:
# BIGGO TO DO:
# trash bin for each filter

In [84]:
# NOTE: Possible names for each sequence

# "3D_T1": ['*SAG*T1*NII', '*SAG*FSPGR*3D*NII']
# "FLAIR": <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_3D_T2_[<type>]<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_T2_[<type>]<DOE>_<tag>.nii
# "BOLD": <ID>_BOLD_<DOE>.nii
# "Diffusion kurtosis": <ID>_{AX}_DkI_<small#dirs>_<DOE>.nii
#                       <ID>_{AX}_DkI_<small#dirs>_<DOE>.bval
#                       <ID>_{AX}_DkI_<small#dirs>_<DOE>.bvec
# .                     <ID>_{AX}_DkI_<big#dirs>_<DOE>.nii
#                       <ID>_{AX}_DkI_<big#dirs>_<DOE>.bval
#                       <ID>_{AX}_DkI_<big#dirs>_<DOE>.bvec
# "Diffusion tensor": <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.nii
#                     <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.bval
#                     <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.bvec
# "Diffusion weithed": <ID>_{AX}_DWI_<DOE>.nii
#                      <ID>_{AX}_DWI_<DOE>.bval
#                      <ID>_{AX}_DWI_<DOE>.bvec
# "ASL": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii   
# "SWAN": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii
# "Carotid"
# "Aorta": <ID>_[CINE]_AORTA_VENC_<number>_<DOE>.nii

In [85]:
def get_sequence_info(sequence_name):
    """
    Convert the input from string to list 
    --------------------------------------------------------
    input:
        sequence_name: string
        
    return:
        array-like: splited the string into items for parsing later
    """
    str_list = sequence_name.replace('-','_').replace('.','_').split('_')
    if not (str_list[-1] == 'NII' or str_list[-1] == 'JSON'):
        return None
    
    str_list = list(filter(None, str_list)) 
    return str_list

#selected everything expect the '.nii' part of the string

In [86]:
def ignore_notes(sequence_info):
    """
    ignore notes inside the parentheses of sequence_info
    --------------------------------------------------------
    input:
        sequence_info: array-like
        
    return:
        array-like: sequence info without the parentheses and notes inside of them
    """
    
    other = []
    
    for part in sequence_info:
        if '(' not in part and ')' not in part:
            other.append(part)

    return other

In [87]:
def get_DOE(info):
    """
    get the date of exam 
    --------------------------------------------------------
    input:
        info: array-like
        
    return:
        string: date of exame in YYYY/MM/DD
    """
    
    if 'I' in info[-1]:
        DOE=info[-2]
        return(DOE[0:4]+"/"+DOE[4:6]+'/'+DOE[6:8])
    else:
        DOE=info[-1]
        return(DOE[0:4]+"/"+DOE[4:6]+'/'+DOE[6:8])

In [88]:
def base_standard(o_name):
    """
    clean up the user input sequence name into a standardized naming convetion for sorting later
        SAGITTLE --> SAG
           AXIAL --> AX
         CORONAL --> COR
            CUBE --> 3D
        WHITTIER --> WH
              RT --> RIGHT
              LT --> LEFT
    --------------------------------------------------------
    input:
        o_name: string
    
    return:
        string: standardized sequnece file name
    """
    
    n_name = o_name.replace('SAGITTAL', 'SAG').replace('CUBE', '3D').replace('AXIAL', 'AX').replace('CORONAL', 'COR')
    n_name = n_name.replace('RT', 'RIGHT').replace('LT', 'LEFT')
    
    info=o_name.replace('-','_').replace('.','_').split('_')
    
    ID = info[0]
    # ID does not follow designation [2-3 letter cohort designation][3-4 number designation]
    match = re.match(r"([A-Z]+)([0-9]+)", ID, re.I)
    if match:
        parts = match.groups()
        if parts[0] == 'WHITTIER':
            n_name = n_name.replace('WHITTIER', 'WH')
    return n_name

In [89]:
# <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_[MIP_ | MIN_IP_]<DOE>[_<tag>].nii
def filt_T1(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    # TO-DO: how to handle 
    # 1) NEUROMELANIN scans
    # 2) SE_MT
    # 3) BRAVO
    # 4) 2DTOF
    # 5) repeat?
    
    sequence_info=get_sequence_info(sequence_name)
    if sequence_info is None:
        return None
    
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1][-1].isdigit():
        trash.append(sequence_name)
        return None

    ignore=['AVG', 'MIN', 'POST_PROCESS', 'MT', '2DTOF', 'OBLIQUE', 'THIN']
    if any(word in sequence_name for word in ignore):
        trash.append(sequence_name)
        return None
    
    accept = ['SAG', 'AX', 'COR']
    if not any(word in sequence_name for word in accept):
        trash.append(sequence_name)
        return None
    
#     print("file name:", file_name)
    return sequence_name[:-4]+'.nii'

    
#TO-DO: repeated exam of the same date or at the different date
    

In [90]:
# Naming to be estabilshed
# <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_[MIP_ | MIN_IP_]<DOE>[_<tag>].nii
def filt_FLAIR(sequence_name, flair_3d, flair_2d, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        flair_3d : array-like
            List of files that are 3D FLAIR
        flair_2d : array-like
            List of files that are 2D FLAIR
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
    if sequence_info is None:
        return None
            
    ignore=['AVG', 'MIN', 'MIP']
    if any(word in sequence_name for word in ignore):
        trash.append(sequence_name)
        return None
    
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None
    
    if "SAG" in sequence_name:
#         print(sequence_info)
        if '3D' in sequence_name:
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    elif 'AX' in sequence_name:
#         print(sequence_info)
        if '3D' in sequence_name:
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    elif 'COR' in sequence_name:
#         print(sequence_info)
        if '3D' in sequence_name:
            flair_3d.append(sequence_name)
        else:
            flair_2d.append(sequence_name)
    else: 
#         print("\t ", sequence_name)
        trash.append(sequence_name)
        return None
            
    sequence_name = '_'.join(sequence_info)+'.nii'
    return sequence_name

#TO-DO: T1 has 3D, 2D

In [91]:
# Naming to be established:
# "T2": <ID>_{SAG | AX | COR}_3D_T2_[<type>]<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_T2_[<type>]<DOE>_<tag>.nii
def filt_T2(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=get_sequence_info(sequence_name)
    if sequence_info is None:
        return None
    
    ignore=['AVG', 'MIN', 'MIP']
    if any(word in sequence_name for word in ignore):
        trash.append(sequence_name)
        return None
    
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None
#     print(sequence_info)
        
    return '_'.join(sequence_info)+'.nii'

#TO-DO: T2 has difference pulse types :0 and some other things

In [92]:
def filt_DWI(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info=sequence_name.replace('-','_').replace('.','_').split('_')
#     print(sequence_info)
    
    
    if 'AX' not in sequence_info:
        sequence_info=sequence_info[0:1]+['AX']+sequence_info[1:]

    if len(sequence_info) > 6:
        trash.append(sequence_name)
#             print(sequence_info)
        return None

    if not sequence_info[-2].isdigit():
        trash.append(sequence_name)
#             print("repeeat:",sequence_info)
        return None
    else:
        #1. concatinate all with _expect the last item
        #2. lower case the extension (last item) and add a . in front of it
        n_name=('_'.join(sequence_info[:-1]))+'.'+sequence_info[-1].lower()
        return n_name

In [93]:
def filt_DTI(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
    n_name = sequence_name.replace('_30_DIRECTION_', '_(30_DIRECTION)_')
    
    n_name = n_name[:-4] + n_name[-4:].lower()
    return n_name

In [94]:
def filt_DKI(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
    if 'EQ_1' in sequence_name:
        trash.append(sequence_name)
        return None
    
    n_name = sequence_name[:-4] + sequence_name[-4:].lower()
    return n_name

In [95]:
# BOLD naming convention: <ID>_BOLD_<DOE>.nii
def filt_bold(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
    problem_bold = None
    
    sequence_info = ignore_notes(sequence_info)
    sequence_name = '_'.join(sequence_info)+'.nii'    
    
    # The following ID's are the problem ones (from previous runs). Checking if these ID's are unique
#     probs = ['SH005', 'WH1296', 'MEN009', 'WH1612', 'WH1268', 'MEN040', 'SH019', 
#              'MEN023', 'MEN007', 'WHITTIER5002']
    
    # If it contains EQ_1 or a letter attached to the end of the DOE,
    # then there is a previous one that already exists
    if sequence_info[-2] == 'EQ' or not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None
        
#     elif ID in probs:
#         problem_bold = sequence_name
    
    elif len(sequence_info) != 3:
        problem_bold = sequence_name
    
#     return problem_bold
    return sequence_name

In [96]:
def filt_asl(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
    sequence_info=ignore_notes(sequence_info)
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None

    ignore=['DELAY', 'FL']
    if any(word in sequence_name for word in ignore):
        trash.append(sequence_name)
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [97]:
def filt_swan(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
    sequence_info=ignore_notes(sequence_info)
    if '3D' not in sequence_info and sequence_info[-1][0] == 'I':
        sequence_info[-1] = sequence_info[-1].lower()
    elif not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None

    ignore=['MIDBRAIN']
    if any(word in sequence_name for word in ignore):
        trash.append(sequence_name)
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [98]:
def filt_csf(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
    if '1.5' in sequence_name:
        trash.append(sequence_name)
        return None
    
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
#     print(sequence_info)
    if not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None
    
    # NOTE: V vs VC vs VENC
    ignore=['CF', 'FM']
    if any(word in sequence_name for word in ignore):
        trash.append(sequence_name)
        return None
    
    return '_'.join(sequence_info)+'.nii'

#TO-DO: CSF of different location and CINE ON/OFF

In [99]:
def filt_carotid(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
#     print(sequence_name)

    if not sequence_info[-1][-1].isdigit():
        trash.append(sequence_name)
        return None 
    
    return '_'.join(sequence_info)+'.nii'

In [100]:
def filt_aorta(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None

    ignore=['ASCENDING']
    
    if any(word in sequence_name for word in ignore):
        trash.append(sequence_name)
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [101]:
def filt_candy_cane(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
#     print(sequence_info)
    
    if not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [102]:
def filt_arch(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
#     print(sequence_name)
#     print(info)
    if not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None
    
    if 'VENC' not in info:
        info.insert(3, 'VENC_???')
    
    return '_'.join(info)[:-4]+'.nii'

In [103]:
def filt_mpr(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
#     print(sequence_name)
    print(sequence_info)

In [104]:
def filt_cbf(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
    if sequence_info[-2] == 'EQ' or not sequence_info[-1].isdigit():
        trash.append(sequence_name)
        return None
    
    return '_'.join(sequence_info)+'.nii'

In [105]:
def filt_probe(sequence_name, trash):
    """
    Determine if file_name passed in should be kept
    --------------------------------------------------------
    Parameters:
        file_name : string
            Name of file to be checked
        trash : array-like
            List of files that are filtered out by the filter
    
    Returns:
        file_name : string or None
            If file should be kept, return the a renamed version of the file
            else if the file is not to be kept, None is returned
    """
    sequence_info = get_sequence_info(sequence_name)
    
    if sequence_info is None:
        return None
    
#     print(sequence_name)
#     print(sequence_info)
    return '_'.join(sequence_info)+'.nii'

In [106]:
def get_path(root):
    """
    Fliter through all the files in the given path
    --------------------------------------------------------
    Parameters:
        root : string
            Path the directory that contains all the files that is to be filtered through
    """
    
    t1=[]
    flair = []
    flair_3d = []
    flair_2d = []
    t2 = []
    bold = []
    dwi=[]
    dti = []
    dki = []
    swan = []
    asl=[]
    candy_cane=[]
    arch=[]
    aorta=[]
    csf = []
    carotid = []
    cbf=[]
    probe = []
        
    t1_t=[]
    flair_t = []
    t2_t = []
    bold_t = []
    dwi_t=[]
    dti_t = []
    dki_t = []
    swan_t = []
    asl_t=[]
    candy_cane_t=[]
    arch_t=[]
    aorta_t=[]
    csf_t = []
    carotid_t = []
    cbf_t=[]
    probe_t = []
    
    change = 0
    count = 0
    
    for path, subdirs, files in os.walk(root):
        for file in files:
            file_upper=base_standard(file.upper())
            if file_upper is None:
                continue
            if fnmatch.fnmatch(file_upper, '*T1*') or fnmatch.fnmatch(file_upper, '*FSPGR*'):                 
                n_file=filt_T1(file_upper, t1_t)
#                 print("T1:", n_file)
                if n_file is not None:
                    t1.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*FLAIR*'): 
#                 print("FLAIR: \t", file_upper)
                n_file = filt_FLAIR(file_upper, flair_3d, flair_2d, flair_t)
#                 print("FLAIR:", n_file)
                if n_file is not None:
                    flair.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*T2*'):
#                 print('\t', file_upper)
                n_file = filt_T2(file_upper, t2_t)
                if n_file is not None:
                    t2.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*BOLD*'):
                n_file = filt_bold(file_upper, bold_t)
                if n_file is not None:
#                     print(file, '-->', n_file)
                    bold.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*DWI*'):
#                 print('\t', file_upper)
                n_file=filt_DWI(file_upper, dwi_t)
                if n_file is not None:
                    dwi.append((file, n_file))
            elif fnmatch.fnmatch(file_upper, '*DTI*'):
#                 print('\t', file_upper)
                n_file=filt_DTI(file_upper, dti_t)
                if n_file is not None:
                    dti.append((file, n_file))
            elif fnmatch.fnmatch(file_upper, '*DKI*'):
#                 print('\t', file_upper)
                n_file=filt_DKI(file_upper, dki_t)
                if n_file is not None:
                    dki.append((file, n_file))
            elif fnmatch.fnmatch(file_upper, '*ASL*'):
                n_file=filt_asl(file_upper, asl_t)
                if n_file is not None:
                    asl.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
#                 pass
            elif fnmatch.fnmatch(file_upper, '*SWAN*'):
                # Swan files has very consistant sequence naming
                # beaware of ID naming though
                n_file = filt_swan(file_upper, swan_t)
                if n_file is not None:
                    swan.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
            elif fnmatch.fnmatch(file_upper, '*CSF*'):
#                 print('CSF:', file_upper)
                n_file = filt_csf(file_upper, csf_t)
                if n_file is not None:
                    csf.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
                pass
            elif fnmatch.fnmatch(file_upper, '*CAROTID*'):
                #Need to ask what the carotid is doing: 
                #1. need to know the naming convention for difference carotids
#                 print('Carotid:', file_upper)
                n_file = filt_carotid(file_upper, carotid_t)
                if n_file is not None:
                    carotid.append((file, n_file))
                pass
            elif fnmatch.fnmatch(file_upper, '*AORTA*'):
                n_file=filt_aorta(file_upper, aorta_t)
                if n_file is not None:
                    aorta.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
            elif fnmatch.fnmatch(file_upper, '*CEREBR*BLOOD*FLOW*'):
#                 print("cbf:", file_upper)
                n_file=filt_cbf(file_upper, cbf_t)
                if n_file is not None:
                    cbf.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                     print(file, '-->', n_file)
            elif fnmatch.fnmatch(file_upper, '*MPR*'):
#                 print(file_upper)
#                 filt_mpr(file_upper)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*CANDY_CANE*'):
#                 print('Candy cane:', file_upper)
                n_file=filt_candy_cane(file_upper, candy_cane_t)
                if n_file is not None:
                    candy_cane.append((file, n_file))
#                     if file != n_file:
#                     print(file, '-->', n_file)
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*ARCH*'):
                #    
                # 1. the inconsidency happens at the missing venc number.
                #
#                 print(file_upper)
                n_file = filt_arch(file_upper, arch_t)
                if n_file is not None:
                    arch.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                     print(file, '-->', n_file)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*PROBE*'):
#                 print(file_upper)
                n_file = filt_probe(file_upper, probe_t)
                if n_file is not None:
                    probe.append((file, n_file))
#                     if file != n_file:
#                         #TODO: rename file
#                         print(file, '-->', n_file)
                pass
            elif fnmatch.fnmatch(file_upper, '*'):
#                 print('\t', file_upper)
                pass
            else:
#                 print('\t\t', file_upper)
                pass

    filtered = {'T1' : t1,
                'FLAIR' : flair,
                'T2' : t2,
                'BOLD' : bold,
                'DWI' : dwi,
                'DTI' : dti,
                'DKI' : dki,
                'SWAN' : swan,
                'ASL' : asl,
                'CANDY CANE' : candy_cane,
                'ARCH' : arch,
                'AORTA' : aorta,
                'CSF': csf,
                'CBF': cbf,
                'PROBE' :probe}

    trash = {'T1' : t1_t,
             'FLAIR' : flair_t,
             'T2' : t2_t,
             'BOLD' : bold_t,
             'DWI' : dwi_t,
             'DTI' : dti_t,
             'DKI' : dki_t,
             'SWAN' : swan_t,
             'ASL' : asl_t,
             'CANDY CANE' : candy_cane_t,
             'ARCH' : arch_t,
             'AORTA' : aorta_t,
             'CSF': csf_t,
             'CBF': cbf_t,
             'PROBE' :probe_t}
#     print(len(t1))
#     for pair in t1:
#         print(pair)
#     print(len(flair))
#     for pair in flair:
#         print(pair)
#     print(len(t2))
#     for pair in t2:
#         print(pair)
#     print(len(bold))
#     for pair in bold:
#         print(pair)
#     print(len(asl))
#     for pair in asl:
#         print(pair)
#     print(len(swan))
#     for pair in swan:
#         print(pair)
#     print(len(csf))
#     for pair in csf:
#         print(pair)
#     print(len(aorta))
#     for pair in aorta:
#         print(pair)
#     print(len(cbf))
#     for pair in cbf:
#         print(pair)
#     print(len(candy_cane))
#     for pair in candy_cane:
#         print(pair)
#     print(len(arch))
#     for pair in arch:
#         print(pair)
#     print(len(probe))
#     for pair in probe:
#         print(pair)

#     print("count: ", count)
#     print(change)

#     for can in trash:
#         print(can)
#         for entry in trash[can]:
#             print('\t', entry)

In [107]:
def main():
#     source_dir=input("Enter source dir:")
    source_dir="/Users/baymac/Desktop/Project_NaCl_Test_DataSet/3T_Last_updated_31OCT19KW"
#     source_dir = '/media/ke/8tb_part2/FSL_work/all_info'
    get_path(source_dir)
        

In [108]:
if __name__ =="__main__":
    main()

T1
	 SH049_AX_FSPGR_3D_T1_20170829093206_I00002.JSON
	 SH049_AX_FSPGR_3D_T1_20170829093206_I00001.NII
	 SH049_AX_FSPGR_3D_T1_20170829093206_I00001.JSON
	 SH049_AX_FSPGR_3D_T1_20170829093206_I00002.NII
	 SH049_SAG_FSPGR_3D_20170829093206.NII
	 SH049_SAG_FSPGR_3D_20170829093206.JSON
	 MEN005_AX_FSPGR_MIN_20170223153214.NII
	 MEN005_AX_FSPGR_MIN_20170223153214.JSON
	 MEN005_COR_FSPGR_AVG_20170223153214.NII
	 MEN005_COR_FSPGR_AVG_20170223153214.JSON
	 MEN005_SAG_FSPGR_AVG_20170223153214.NII
	 MEN005_SAG_FSPGR_AVG_20170223153214.JSON
	 MEN005_SAG_FSPGR_3D_20170223153214.NII
	 MEN005_SAG_FSPGR_3D_20170223153214.JSON
	 WH1331_SAG_T1_3D_20190514132222.JSON
	 WH1331_SAG_T1_3D_20190514132222.NII
	 WH1331_COR_T1_20190514132222.JSON
	 WH1331_COR_T1_20190514132222.NII
	 WH1331_SAG_T1_20190514132222.NII
	 WH1331_SAG_T1_20190514132222.JSON
	 WH1331_AX_T1_20190514132222.JSON
	 WH1331_AX_T1_20190514132222.NII
	 MEN021_AX_FSPGR_MIN_20170302095957.JSON
	 MEN021_AX_FSPGR_MIN_20170302095957.NII
	 MEN021_CO

	 BR038_AX_FSPGR_3D_T1_20181026104213_I00002.NII
	 BR038_SAG_FSPGR_3D_20181026104213.JSON
	 BR038_SAG_FSPGR_3D_20181026104213.NII
	 BR043_AX_T1_FSPGR_3D_20190205131226_I00002.NII
	 BR043_AX_T1_FSPGR_3D_20190205131226_I00002.JSON
	 BR043_AX_T1_FSPGR_3D_20190205131226_I00001.JSON
	 BR043_AX_T1_FSPGR_3D_20190205131226_I00001.NII
	 BR043_SAG_FSPGR_3D_20190205131226.NII
	 BR043_SAG_FSPGR_3D_20190205131226.JSON
	 MEN051_AX_T1_FSPGR_3D_20190220154013_I00001.NII
	 MEN051_AX_T1_FSPGR_3D_20190220154013_I00002.JSON
	 MEN051_AX_T1_FSPGR_3D_20190220154013_I00002.NII
	 MEN051_AX_T1_FSPGR_3D_20190220154013_I00001.JSON
	 MEN050_SAG_FSPGR_3D_20190220154013.JSON
	 MEN050_SAG_FSPGR_3D_20190220154013.NII
	 WH1657_AX_FSPGR_3D_T1_20180409150847_I00001.JSON
	 WH1657_AX_FSPGR_3D_T1_20180409150847_I00001.NII
	 WH1657_AX_FSPGR_3D_T1_20180409150847_I00002.NII
	 WH1657_AX_FSPGR_3D_T1_20180409150847_I00002.JSON
	 WH1657_COR_FSPGR_3D_T1_20180409150847_I00002.NII
	 WH1657_COR_FSPGR_3D_T1_20180409150847_I00002.JSON
	

	 SH016_AX_FSPGR_3D_T1_20170510161050_I00001.NII
	 SH016_AX_FSPGR_3D_T1_20170510161050_I00002.NII
	 SH016_AX_FSPGR_3D_T1_20170510161050_I00001.JSON
	 SH016_AX_FSPGR_3D_T1_20170510161050_I00002.JSON
	 SH016_SAG_FSPGR_3D_20170510161050.JSON
	 SH016_SAG_FSPGR_3D_20170510161050.NII
	 MEN043_AX_FSPGR_3D_T1_20180215095842_I00002.JSON
	 MEN043_AX_FSPGR_3D_T1_20180215095842_I00001.JSON
	 MEN043_AX_FSPGR_3D_T1_20180215095842_I00002.NII
	 MEN043_AX_FSPGR_3D_T1_20180215095842_I00001.NII
	 MEN043_COR_FSPGR_3D_T1_20180215095842_I00001.JSON
	 MEN043_COR_FSPGR_3D_T1_20180215095842_I00002.JSON
	 MEN043_COR_FSPGR_3D_T1_20180215095842_I00001.NII
	 MEN043_COR_FSPGR_3D_T1_20180215095842_I00002.NII
	 MEN043_SAG_FSPGR_3D_20180215095842.NII
	 MEN043_SAG_FSPGR_3D_20180215095842.JSON
	 SH056_AX_FSPGR_3D_T1_20180523111443_I00001.JSON
	 SH056_AX_FSPGR_3D_T1_20180523111443_I00002.JSON
	 SH056_AX_FSPGR_3D_T1_20180523111443_I00001.NII
	 SH056_AX_FSPGR_3D_T1_20180523111443_I00002.NII
	 SH056_AX_T1_SE_MT_201805231114

	 WH1354_SAG_3D_FLAIR_20180608161238.NII
	 WH1354_COR_FLAIR_20180608161238_I00001.NII
	 WH1354_COR_FLAIR_20180608161238_I00002.JSON
	 WH1354_COR_FLAIR_20180608161238_I00001.JSON
	 WH1354_COR_FLAIR_20180608161238_I00002.NII
	 WH1354_AX_FLAIR_20180608161238_I00001.NII
	 WH1354_AX_FLAIR_20180608161238_I00002.JSON
	 WH1354_AX_FLAIR_20180608161238_I00001.JSON
	 WH1354_AX_FLAIR_20180608161238_I00002.NII
	 WH1310_SAG_3D_FLAIR_20170503101521.NII
	 WH1310_SAG_3D_FLAIR_20170503101521.JSON
	 WH1310_AX_FLAIR_20170503101521_I00002.JSON
	 WH1310_AX_FLAIR_20170503101521_I00002.NII
	 WH1310_AX_FLAIR_20170503101521_I00001.NII
	 WH1310_AX_FLAIR_20170503101521_I00001.JSON
	 SH035_SAG_3D_FLAIR_20170509143850.JSON
	 SH035_SAG_3D_FLAIR_20170509143850.NII
	 SH017_AX_3D_FLAIR_RECON_AVG_20170213140013.NII
	 SH017_AX_3D_FLAIR_RECON_AVG_20170213140013.JSON
	 SH017_SAG_3D_FLAIR_20170213140013.JSON
	 SH017_SAG_3D_FLAIR_20170213140013.NII
	 SH017_AX_3D_FLAIR_MIP_20170213140013.NII
	 SH017_AX_3D_FLAIR_MIP_2017021314

	 WH1696_AX_T2_FLAIR_20181101112837.JSON
	 WH1523_SAG_3D_FLAIR_20170509110534.NII
	 WH1523_SAG_3D_FLAIR_20170509110534.JSON
	 WH1523_COR_FLAIR_20170509110534_I00001.NII
	 WH1523_COR_FLAIR_20170509110534_I00001.JSON
	 WH1523_COR_FLAIR_20170509110534_I00002.JSON
	 WH1523_COR_FLAIR_20170509110534_I00002.NII
	 WH1523_AX_FLAIR_20170509110534_I00002.NII
	 WH1523_AX_FLAIR_20170509110534_I00002.JSON
	 WH1523_AX_FLAIR_20170509110534_I00001.NII
	 WH1523_AX_FLAIR_20170509110534_I00001.JSON
	 SH047_SAG_3D_FLAIR_20170712104739.NII
	 SH047_SAG_3D_FLAIR_20170712104739.JSON
	 MEN017_AX_3D_FLAIR_RECON_AVG_20170224140230.NII
	 MEN017_AX_3D_FLAIR_RECON_AVG_20170224140230.JSON
	 MEN017_SAG_3D_FLAIR_20170224140230.JSON
	 MEN017_SAG_3D_FLAIR_20170224140230.NII
	 MEN017_AX_3D_FLAIR_MIP_20170224140230.NII
	 MEN017_AX_3D_FLAIR_MIP_20170224140230.JSON
	 MEN017_SAG_3D_FLAIR_MIP_20170224140230.JSON
	 MEN017_SAG_3D_FLAIR_MIP_20170224140230.NII
	 MEN050_SAG_3D_FLAIR_20190220151037.NII
	 MEN050_SAG_3D_FLAIR_20190220

	 BR035_SAG_3D_T2_20190702101108.NII
	 WH1280_AX_T2_20190514094350.JSON
	 WH1280_AX_T2_20190514094350.NII
	 WH1280_T2-WEIGHTED_TRACE_20190514094350.JSON
	 WH1280_T2-WEIGHTED_TRACE_20190514094350.NII
	 WH1280_COR_T2_20190514094350.NII
	 WH1280_COR_T2_20190514094350.JSON
	 WH1280_SAG_T2_20190514094350.JSON
	 WH1280_SAG_T2_20190514094350.NII
	 WH1280_SAG_3D_T2_20190514094350.NII
	 WH1280_SAG_3D_T2_20190514094350.JSON
	 WH1741_SAG_T2_3D_20190626135539.NII
	 WH1741_SAG_T2_3D_20190626135539.JSON
	 WH1741_AX_T2_20190626135539.JSON
	 WH1741_AX_T2_20190626135539.NII
	 WH1741_COR_T2_20190626135539.NII
	 WH1741_COR_T2_20190626135539.JSON
	 WH1741_SAG_T2_20190626135539.JSON
	 WH1741_SAG_T2_20190626135539.NII
	 BR046_SAG_3D_T2_20190530091459.NII
	 BR046_SAG_3D_T2_20190530091459.JSON
	 MEN009_AX_3D_T2_MIP_20170307100421.JSON
	 MEN009_AX_3D_T2_MIP_20170307100421.NII
	 MEN009_AX_3D_T2_AVG_20170307100421.JSON
	 MEN009_AX_3D_T2_AVG_20170307100421.NII
	 MEN009_AX_3D_T2_MIN_IP_20170307100421.JSON
	 MEN009

	 SH028_BOLD_20170512104944_NII.nii
	 WH1736_BOLD_20190708103024_JSON.nii
	 WH1736_BOLD_20190708103024_NII.nii
	 WH1646_BOLD_20171215105227_NII.nii
	 WH1646_BOLD_20171215105227_JSON.nii
	 SH045_BOLD_20170706133844_JSON.nii
	 SH045_BOLD_20170706133844_NII.nii
	 MEN038_BOLD_20170418135909_JSON.nii
	 MEN038_BOLD_20170418135909_NII.nii
	 SH025_BOLD_20170329141311_JSON.nii
	 SH025_BOLD_20170329141311_NII.nii
	 MEN040_BOLD_20170512130925_NII.nii
	 MEN040_BOLD_20170512130925_JSON.nii
	 BR009_BOLD_20171030160357_JSON.nii
	 BR009_BOLD_20171030160357_NII.nii
	 SH031_BOLD_20170505142808_NII.nii
	 SH031_BOLD_20170505142808_JSON.nii
	 BR024_BOLD_20180227163900_NII.nii
	 BR024_BOLD_20180227163900_JSON.nii
	 SH060_BOLD_20190820133401_NII.nii
	 SH060_BOLD_20190820133401_JSON.nii
	 WH1406_BOLD_20170511142945_JSON.nii
	 WH1406_BOLD_20170511142945_NII.nii
	 MEN052_BOLD_20190318124108_JSON.nii
	 MEN052_BOLD_20190318124108_NII.nii
	 WH1349_BOLD_20180627143541_JSON.nii
	 WH1349_BOLD_20180627143541_NII.nii
	

	 WH1327_AX_ASL_(NON-CONTRAST)_20180508145106.NII
	 WH1327_AX_ASL_(NON-CONTRAST)_20180508145106.JSON
	 WH1686_AX_3D_ASL_(NON-CONTRAST)_20181010154013.NII
	 WH1686_AX_3D_ASL_(NON-CONTRAST)_20181010154013.JSON
	 WH1686_AX_3D_ASL_(NON-CONTRAST)_20181010154013A.NII
	 WH1686_AX_3D_ASL_(NON-CONTRAST)_20181010154013A.JSON
	 BR003_3D_ASL_(NON-CONTRAST)_20170620095427A.NII
	 BR003_3D_ASL_(NON-CONTRAST)_20170620095427.NII
	 BR003_3D_ASL_(NON-CONTRAST)_20170620095427A.JSON
	 BR003_3D_ASL_(NON-CONTRAST)_20170620095427.JSON
	 WH1354_AX_ASL_(NON-CONTRAST)_20180608161238A.NII
	 WH1354_AX_ASL_(NON-CONTRAST)_20180608161238.NII
	 WH1354_AX_ASL_(NON-CONTRAST)_20180608161238.JSON
	 WH1354_AX_ASL_(NON-CONTRAST)_20180608161238A.JSON
	 WH1310_3D_ASL_(NON-CONTRAST)_20170503101521A.JSON
	 WH1310_3D_ASL_(NON-CONTRAST)_20170503101521A.NII
	 WH1310_3D_ASL_(NON-CONTRAST)_20170503101521.JSON
	 WH1310_3D_ASL_(NON-CONTRAST)_20170503101521.NII
	 BR016_AX_ASL_(NON-CONTRAST)_20171207130232.JSON
	 BR016_AX_ASL_(NON-CONTR

	 WH1741_CINE_ARCH_VENC_150_20190626135539.NII
	 WH1741_CINE_ARCH_VENC_150_20190626135539.JSON
	 BR046_CINE_ARCH_VENC_150_20190530091459.JSON
	 BR046_CINE_ARCH_VENC_150_20190530091459.NII
	 SH003_CINE_ARCH_20190816130333A.JSON
	 SH003_CINE_ARCH_20190816130333.JSON
	 SH003_CINE_ARCH_20190816130333.NII
	 SH003_CINE_ARCH_20190816130333A.NII
	 WH1727_CINE_ARCH_VENC_150_20190603131028A.JSON
	 WH1727_CINE_ARCH_VENC_150_20190603131028.JSON
	 WH1727_CINE_ARCH_VENC_150_20190603131028.NII
	 WH1727_CINE_ARCH_VENC_150_20190603131028A.NII
	 BR051_CINE_ARCH_20190909111639.JSON
	 BR051_CINE_ARCH_20190909111639.NII
	 WH1275_CINE_ARCH_VENC_150_20190419091312.NII
	 WH1275_CINE_ARCH_VENC_150_20190419091312.JSON
	 WH1275_CINE_ARCH_VENC_150_20190419091312A.NII
	 WH1275_CINE_ARCH_VENC_150_20190419091312A.JSON
	 WH1738_CINE_ARCH_20190715094319.NII
	 WH1738_CINE_ARCH_20190715094319.JSON
	 WH1736_CINE_ARCH_20190708103024.JSON
	 WH1736_CINE_ARCH_20190708103024.NII
	 SH060_CINE_ARCH_20190820133401.NII
	 SH060_CI

In [ ]:
# NOTE: Standarized Naming
# > study ID: 2 letters 

# "3D_T1": <AB####>
# "FLAIR": <ID>_{SAG | AX | COR}_[T1_ | T2_][3D_]FLAIR_<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_3D_T2_[<type>]<DOE>.nii
# "T2": <ID>_{SAG | AX | COR}_T2_[<type>]<DOE>_<tag>.nii
# "BOLD": <ID>_BOLD_<DOE>.nii
# "Diffusion kurtosis": <ID>_{AX}_DkI_<small#dirs>_<DOE>.nii
#                       <ID>_{AX}_DkI_<small#dirs>_<DOE>.bval
#                       <ID>_{AX}_DkI_<small#dirs>_<DOE>.bvec
# .                     <ID>_{AX}_DkI_<big#dirs>_<DOE>.nii
#                       <ID>_{AX}_DkI_<big#dirs>_<DOE>.bval
#                       <ID>_{AX}_DkI_<big#dirs>_<DOE>.bvec
# "Diffusion tensor": <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.nii
#                     <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.bval
#                     <ID>_{AX}_D{k|T}I_<#dir>_<DOE>.bvec
# "Diffusion weithed": <ID>_{AX}_DWI_<DOE>.nii
#                      <ID>_{AX}_DWI_<DOE>.bval
#                      <ID>_{AX}_DWI_<DOE>.bvec
# "ASL": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii   
# "SWAN": <ID>_{SAG | AX | COR}_ASL_<DOE>.nii
# "Carotid"
# "Aorta": <ID>_[CINE]_AORTA_VENC_<number>_<DOE>.nii